In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_lattice as tfl

In [2]:
# Import helpers
import import_ipynb
from common import buildDatasetForLattice
from common import scaleVolume
from common import extractLatticeWeights
from common import dropColumns
from common import filterBad
from common import splitDataset
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractXGWeights

importing Jupyter notebook from common.ipynb


In [3]:
only_use_strong = 54

dataset, columns = buildDatasetForLattice()
dataset = filterBad(dataset, only_use_strong)
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

Rejected 6196746 points (56.847699%)


In [4]:
dataset = dataset.drop(['row'], axis=1)
columns = [x for x in columns if x != 'row']
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

In [ ]:
def buildLattice(num_keypoints, lattice_size, columns):
    # Use ParallelCombination helper layer to group togehter calibration layers
    # which have to be executed in paralel in order to be able to use Sequential
    # model. Alternatively you can use functional API.
    combined_calibrators = tfl.layers.ParallelCombination()

    # Configure calibration layers for every feature:

    for ft in columns:
        kpts = np.percentile(dataset[ft], np.linspace(0,99,num_keypoints))
        for i in range(1,len(kpts)):
            while kpts[i] <= kpts[i-1]:
                kpts[i] = kpts[i] + 1
        calibrator = tfl.layers.PWLCalibration(
            # Every PWLCalibration layer must have keypoints of piecewise linear
            # function specified. Easiest way to specify them is to uniformly cover
            # entire input range by using numpy.linspace().
            input_keypoints=kpts,
#             input_keypoints=np.linspace(dataset[ft].min(),
#                                       dataset[ft].max(),
#                                       num=num_keypoints),
        # You need to ensure that input keypoints have same dtype as layer input.
        # You can do it by setting dtype here or by providing keypoints in such
        # format which will be converted to deisred tf.dtype by default.
        dtype=tf.float32,
        # Output range must correspond to expected lattice input range.
        output_min=0.0,
        output_max=lattice_size - 1.0,
        monotonicity='increasing')
        combined_calibrators.append(calibrator)

    # Create Lattice layer to nonlineary fuse output of calibrators. Don't forget
    # to specify monotonicity 'increasing' for any dimension which calibrator is
    # monotonic regardless of monotonicity direction of calibrator. This includes
    # partial monotonicity of CategoricalCalibration layer.
    lattice = tfl.layers.Lattice(
      lattice_sizes=[lattice_size for _ in columns],
      monotonicities=['increasing' if (ft == 'rising_idx' or ft == 'falling_idx') else 'none' for x in columns],
      output_min=dataset['delay'].min(),
      output_max=dataset['delay'].max())

    model = keras.models.Sequential()
    # We have just 2 layer as far as Sequential model is concerned.
    # PWLConcatenate layer takes care of grouping calibrators.
    model.add(combined_calibrators)
    model.add(lattice)
    model.compile(loss=keras.losses.mean_squared_error,
                optimizer=keras.optimizers.Adagrad(learning_rate=80.0))
    return model




In [ ]:
def trainLattice(model, modelName, cols, epochs):
    
    features = train_dataset[cols].values.astype(np.float32)
    target = train_labels.values.astype(np.float32)

    checkpoint = ModelCheckpoint('./' + modelName, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
    model.fit(features,
            target,
            batch_size=32,
            epochs=epochs,
            validation_split=0.05,
            shuffle=False,
            callbacks=[checkpoint])
    return model

In [ ]:
show_perf = True
load_prior_model = False
train_model = True
strong_data_sfx = "_strong" if (only_use_strong > 0) else ""

Hypercube_8_3 = buildLattice(8,3,columns[:-1])
if (load_prior_model): 
    Hypercube_8_3.build((None,4,))
    Hypercube_8_3.load_weights('./Hypercube_8_3' + strong_data_sfx) #load
if (train_model): 
    Hypercube_8_3 = trainLattice(Hypercube_8_3,'Hypercube_8_3' + strong_data_sfx,columns[:-1], 30) #train
    Hypercube_8_3.save('./Hypercube_8_3' + strong_data_sfx)
preds = Hypercube_8_3.predict(test_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractLatticeWeights(Hypercube_8_3, 'Hypercube_8_3' + strong_data_sfx)
print('--------------------------------')


Hypercube_4_5 = buildLattice(4,5,columns[:-1])
if (load_prior_model): 
    Hypercube_4_5.build((None,4,))
    Hypercube_4_5.load_weights('./Hypercube_4_5' + strong_data_sfx) #load
if (train_model): 
    Hypercube_4_5 = trainLattice(Hypercube_4_5,'Hypercube_4_5' + strong_data_sfx,columns[:-1], 30) #train
    Hypercube_4_5.save('./Hypercube_4_5' + strong_data_sfx)
preds = Hypercube_4_5.predict(test_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractLatticeWeights(Hypercube_4_5, 'Hypercube_4_5' + strong_data_sfx)
print('--------------------------------')



Train on 3574936 samples, validate on 188155 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/30
3574936/3574936 [==============================] - 999s 280us/sample - loss: 184335.0286 - val_loss: 333961.7230
Epoch 2/30
3574936/3574936 [==============================] - 920s 257us/sample - loss: 66570.6110 - val_loss: 148608.1475
Epoch 3/30
3574936/3574936 [==============================] - 798s 223us/sample - loss: 64445.9225 - val_loss: 35954.9167
Epoch 4/30
3574936/3574936 [==============================] - 961s 269us/sample - loss: 58786.6541 - val_loss: 19518.0023
Epoch 5/30
3574936/3574936 [==============================] - 972s 272us/sample - loss: 39986.8977 - val_loss: 32861.9629
Epoch 6/30
3574936/3574936 [==============================] - 948s 265us/sample - loss: 36209.0206 - val_loss: 27317.4193


3574936/3574936 [==============================] - 591s 165us/sample - loss: 8684.1765 - val_loss: 1928.0091
Epoch 28/30
3574936/3574936 [==============================] - 591s 165us/sample - loss: 8156.7809 - val_loss: 7105.8565
Epoch 29/30
3574936/3574936 [==============================] - 591s 165us/sample - loss: 8050.1412 - val_loss: 6081.2197
Epoch 30/30
3574936/3574936 [==============================] - 590s 165us/sample - loss: 7765.3506 - val_loss: 6354.7730
MSE, MAE, r2: 6437.035350,68.816056,0.993856
--------------------------------
TBD :)
--------------------------------
Train on 3574936 samples, validate on 188155 samples
Epoch 1/30
3574936/3574936 [==============================] - 751s 210us/sample - loss: 122758.0816 - val_loss: 3628.0072
Epoch 2/30
3574936/3574936 [==============================] - 747s 209us/sample - loss: 2462.8993 - val_loss: 1445.5760
Epoch 3/30
3574936/3574936 [==============================] - 735s 205us/sample - loss: 1857.3399 - val_loss: 1390.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 928992/3574936 [======>.......................] - ETA: 8:23 - loss: 1308.0957

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

3574936/3574936 [==============================] - 695s 194us/sample - loss: 1094.1867 - val_loss: 1050.5199
Epoch 24/30
 188288/3574936 [>.............................] - ETA: 10:51 - loss: 1192.4616

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1449152/3574936 [===========>..................] - ETA: 6:46 - loss: 1095.0246

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3574936/3574936 [==============================] - 703s 197us/sample - loss: 1078.7014 - val_loss: 1054.5994
Epoch 25/30
1238592/3574936 [=========>....................] - ETA: 7:28 - loss: 1081.4934

In [ ]:
preds = Hypercube_8_4.predict(test_dataset.values.astype(np.float32))

In [21]:
if (show_perf): evaluatePerf(test_labels, preds)

MSE, MAE, r2: 19563.272052 51.622984 0.988916


In [88]:
# print(Hypercube_8_2)
# print(train_dataset.values)
# print(train_labels.values)
test = buildLattice(2,8,columns[:-1])
print(test)
# print(Hypercube_8_2.summary())

In [96]:
features = train_dataset[columns[:-1]].values.astype(np.float32)[0:10]
print(features)
target = train_labels.values.astype(np.float32)[0:10]

test.fit(features,
        target,
        batch_size=32,
        epochs=1,
        validation_split=0.05,
        shuffle=False)

[[1.200e+01 1.900e+02 3.790e+02 1.300e+01 0.000e+00]
 [3.600e+01 2.480e+02 4.710e+02 1.943e+03 0.000e+00]
 [2.500e+01 4.000e+01 2.010e+02 2.500e+01 0.000e+00]
 [9.600e+01 1.390e+02 3.180e+02 1.000e+00 0.000e+00]
 [6.600e+01 4.000e+01 1.470e+02 0.000e+00 0.000e+00]
 [2.400e+01 1.470e+02 3.270e+02 1.850e+02 0.000e+00]
 [1.400e+01 1.550e+02 3.370e+02 4.890e+02 0.000e+00]
 [1.050e+02 1.850e+02 3.730e+02 1.200e+01 0.000e+00]
 [4.900e+01 7.000e+01 8.300e+01 3.660e+03 0.000e+00]
 [2.600e+01 5.200e+01 2.150e+02 5.900e+01 0.000e+00]]
Train on 9 samples, validate on 1 samples
9/9 [==============================] - 1s 137ms/sample - loss: 1308289.0000 - val_loss: 2232724.7500


In [98]:
print(test.summary())
preds = test.predict(features)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
parallel_combination_21 (Par multiple                  40        
_________________________________________________________________
lattice_21 (Lattice)         multiple                  32        
Total params: 72
Trainable params: 72
Non-trainable params: 0
_________________________________________________________________
None


In [99]:
print(preds)

[[ -84.66176]
 [ 204.66684]
 [ -94.94145]
 [-149.71718]
 [-133.84915]
 [ -71.80316]
 [ -34.44699]
 [-163.96371]
 [ 572.02185]
 [ -91.2105 ]]


In [100]:
test.save('./test')

In [103]:
test2 = buildLattice(2,8,columns[:-1])
test2.build((None,5,))
test2.load_weights('./test')

In [104]:
print(test2.predict(features))

[[ -84.66176]
 [ 204.66684]
 [ -94.94145]
 [-149.71718]
 [-133.84915]
 [ -71.80316]
 [ -34.44699]
 [-163.96371]
 [ 572.02185]
 [ -91.2105 ]]


In [85]:
preds = Hypercube_8_2.predict(test_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)

IndexError: list index out of range